In [ ]:
# pull data
# select columns 
# set date as index for each df
# combine dataframes
# make new column with difference
# drop original columns
# would be helpful to model the components
# model the data
# fit and get predictions
# find mse

# write it in a function
# could write a function for each

#### Importing Data and Formatting Dataframe

In [ ]:
import pandas as pd
import 
import quandl
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima_model import ARMA
import statsmodels.api as sm

%matplotlib inline

In [ ]:
cb_iy = quandl.get("ML/USEY", authtoken="aG9j5L1aZEe7Gti-5m1D")
tyc_r = quandl.get("USTREASURY/YIELD", authtoken="aG9j5L1aZEe7Gti-5m1D")